In [1]:
import tensorflow as tf
#import tensorflow.compat.v1 as tf
# tf.enable_eager_execution()
# tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import os
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
%matplotlib inline

In [2]:
data =  pd.read_csv("../train.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data_label =  pd.read_csv("../train_label.csv")
label_dict =  dict([(i,a) for i,a in zip(data_label['SAMPLE_EMAIL'], data_label['DIAG_NM'])])
data['DIAG_NM'] = data['EMAIL'].map(label_dict)
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [3]:
drop_cols = ['timezone',
             'activity_class_5min', 'activity_met_1min',
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']

convert_cols = data.iloc[:, 60:65].columns

In [4]:
processed_data = data.drop(drop_cols, axis=1).copy()
processed_data = processed_data.drop(convert_cols, axis=1).copy()
processed_data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,...,sleep_score,sleep_score_alignment,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta,DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,14346,0,417,0,545,...,62,100,41,50,27,97,66,62,-0.12,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,6352,0,473,0,392,...,54,100,49,43,46,91,35,44,-0.32,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,7297,0,586,0,362,...,61,100,56,47,34,89,41,62,0.07,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,491,0,176,0,34,...,55,100,35,40,44,86,29,56,-0.41,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,5861,0,646,0,377,...,56,100,64,55,39,14,54,59,-0.27,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,3863,3,735,2,155,...,76,100,53,46,74,97,84,82,-0.16,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,4411,1,780,4,167,...,77,100,34,57,76,81,95,86,-0.09,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,11057,1,518,1,332,...,65,100,1,56,69,86,89,64,0.41,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,5135,4,578,0,173,...,81,100,9,59,86,89,98,97,0.27,CN


In [18]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# processed_data -> input_data 
input_data = processed_data.copy()
# y 변수와 X 변수 분할 관리 
input_data_y = input_data['DIAG_NM'].copy()
input_data_X = input_data.drop(['DIAG_NM'], axis=1)

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y = le.fit_transform(list(input_data_y))
print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


In [20]:
input_data_X = input_data_X.iloc[:,2:]
input_data_X

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_rmssd,sleep_score,sleep_score_alignment,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta
0,1.71875,730,2944,14346,0,417,0,545,47,0,...,20,62,100,41,50,27,97,66,62,-0.12
1,1.40625,342,2449,6352,0,473,0,392,8,0,...,19,54,100,49,43,46,91,35,44,-0.32
2,1.46875,401,2544,7297,0,586,0,362,24,0,...,26,61,100,56,47,34,89,41,62,0.07
3,0.34375,27,1850,491,0,176,0,34,1,0,...,30,55,100,35,40,44,86,29,56,-0.41
4,1.46875,333,2518,5861,0,646,0,377,6,0,...,23,56,100,64,55,39,14,54,59,-0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.34375,227,2316,3863,3,735,2,155,17,21,...,29,76,100,53,46,74,97,84,82,-0.16
9323,1.34375,249,2351,4411,1,780,4,167,22,7,...,18,77,100,34,57,76,81,95,86,-0.09
9324,1.53125,570,2682,11057,1,518,1,332,61,7,...,15,65,100,1,56,69,86,89,64,0.41
9325,1.34375,295,2331,5135,4,578,0,173,30,28,...,17,81,100,9,59,86,89,98,97,0.27


In [38]:
tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data_X, 
    input_data_y, 
    test_size = 0.2, 
    random_state = 42,
    shuffle=True,
    stratify = input_data_y # Classification 중요 option
)

In [65]:
from sklearn.preprocessing import StandardScaler
Rscaler = StandardScaler()
result = Rscaler.fit_transform(tr_val_X)
pd.options.display.float_format = '{:.3f}'.format
input_data_X_scaled = pd.DataFrame(result, columns=tr_val_X.columns)
tr_val_X=input_data_X_scaled

result = Rscaler.fit_transform(test_X)
pd.options.display.float_format = '{:.3f}'.format
test_X = pd.DataFrame(result, columns=test_X.columns)

In [149]:
X = tf.keras.layers.Input(shape=[48])
H1 = tf.keras.layers.Dense(48, activation ='relu')(X)
H2 = tf.keras.layers.Dense(48, activation ='relu')(H1)
H3 = tf.keras.layers.Dense(48, activation ='relu')(H2)
H4 = tf.keras.layers.Dense(21, activation ='relu')(H3)
H5 = tf.keras.layers.Dense(11, activation ='relu')(H4)
H6 = tf.keras.layers.Softmax(H5)

y = tf.keras.layers.Dense(1, activation ='softmax')(H5)
model = tf.keras.models.Model(X,y)
model.compile(loss='mse')
model.summary()

model.fit(tr_val_X,tr_val_y,epochs=150,verbose=0)

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <tensorflow.python.keras.layers.advanced_activations.Softmax object at 0x0000017F0AF91550>

In [144]:
a = model.predict(tr_val_X)
b = model.predict(test_X)
print(a)

# anw = []
# anw_t = []
# for i in range(len(a)):
#     if a[i][0] > a[i][1] and a[i][0] >=a[i][2]:
#         anw.append(0)
#     elif a[i][1] >=a[i][0] and a[i][1] > a[i][2]:
#         anw.append(1)     
#     elif a[i][2] > a[i][0] and a[i][2] >= a[i][1]:
#         anw.append(2)     

# for i in range(len(b)):
#     if b[i][0] > b[i][1] and b[i][0] >=b[i][2]:
#         anw_t.append(0)
#     elif b[i][1] >=b[i][0] and a[i][1] > b[i][2]:
#         anw_t.append(1)     
#     elif b[i][2] > b[i][0] and a[i][2] >= b[i][1]:
#         anw_t.append(2)   

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [128]:
print(model.predict(tr_val_X))
print(model.predict(test_X))

[[ 2.2624705   2.2438114   2.2427015 ]
 [-0.02978028 -0.03575972 -0.02957259]
 [ 2.3796093   2.3732047   2.3745666 ]
 ...
 [ 0.02651371  0.01911746  0.02566072]
 [ 2.008056    1.9918703   2.0218487 ]
 [ 0.08797845  0.0916155   0.09393664]]
[[ 0.44746694  0.45051762  0.45134202]
 [-0.1940099  -0.19102846 -0.18796445]
 [ 1.0749235   1.0691022   1.0707768 ]
 ...
 [ 1.319142    1.3165199   1.316947  ]
 [ 0.92523485  0.93226564  0.925318  ]
 [ 1.5799631   1.574483    1.5845349 ]]


In [129]:
a=0
for i in range(len(anw)) : 
    if anw[i] == tr_val_y[i]:
        a += 1
abs(a/len(anw))

0.38292454094625383

In [ ]:
#사람 별로 묶어주기
group_pd  = processed_data.groupby(processed_data["EMAIL"])
group_pd = group_pd.mean()
group_pd['DIAG_NM'] = group_pd.index.map(label_dict)
group_pd = group_pd.reset_index()
group_pd = group_pd.drop('EMAIL', axis = 1).copy()
group_pd